In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)


In [ ]:
def prepare_X(X):
    return (X.astype('float32') - 127.5) / 255

fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()

x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

print(np.max(x_train), np.min(x_train))

In [ ]:
# x = (x - u) / std

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_valid_scaled = scaler.fit_transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.fit_transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

print(np.max(x_train_scaled), np.min(x_train_scaled))

In [ ]:
# tf.keras.models.Sequential()

model = keras.models.Sequential([
        keras.layers.Flatten(input_shape = [28, 28]),
        keras.layers.Dense(300, activation = 'relu'),
        keras.layers.Dense(100, activation = 'relu'),
        keras.layers.Dense(10, activation = 'softmax')
    ])

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'sgd',
              metrics = ["accuracy"])

In [ ]:
history = model.fit(x_train_scaled, y_train, epochs = 10,
                    validation_data = (x_valid_scaled, y_valid))

In [ ]:
def plot_lerning_curves(history):
    pd.DataFrame(history.history).plot(figsize = (8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()

plot_lerning_curves(history)

In [ ]:
model.evaluate(x_test_scaled, y_test)

In [ ]:
tf.saved_model.save(model, "../temp/outputs/09_03_tf_keras_save_model/keras_saved_graph")

In [ ]:
!saved_model_cli show --dir ../temp/outputs/09_03_tf_keras_save_model/keras_saved_graph --all

In [ ]:
!saved_model_cli show --dir ../temp/outputs/09_03_tf_keras_save_model/keras_saved_graph \
    --tag_set serve --signature_def serving_default

In [ ]:
!saved_model_cli run --dir  ../temp/outputs/09_03_tf_keras_save_model/keras_saved_graph \
    --tag_set serve --signature_def serving_default \
    --input_exprs 'flatten_input=np.ones((2, 28, 28))'

In [ ]:
loaded_saved_model = tf.saved_model.load('../temp/outputs/09_03_tf_keras_save_model/keras_saved_graph')
print(list(loaded_saved_model.signatures.keys()))

In [ ]:
inference = loaded_saved_model.signatures['serving_default']
print(inference)

In [ ]:
print(inference.structured_outputs)

In [ ]:
results = inference(tf.constant(x_test_scaled[0:1]))
print(results['dense_2'])